# ĐỒ ÁN CUỐI KÌ

Bùi Hữu Nghĩa - 1712613

## TỔNG QUAN

Qui trình KHDL:
- Đưa ra câu hỏi
- Thu thập dữ liệu
- Khám phá dữ liệu
- Tiền xử lí
- Phân tích, trả lời câu hỏi

## 1 - THU THẬP DỮ LIỆU

Ở đây em có dữ liệu là thông tin về các cuốn sách như tên, ngôn ngữ, rating,...  
Dữ liệu được lấy từ trang [kaggle.com](https://www.kaggle.com)  
Thông tin về dữ liệu tại [đây](https://www.kaggle.com/bahramjannesarr/goodreads-book-datasets-10m?select=book1-100k.csv)  
Người sở hữu: Bahram Jannesar & Soroush Ghaderi  
License: **CC0: Public Domain**  
Tác giả thu thập dữ liệu ở trang [goodreads.com](https://www.goodreads.com/)

## 2 - KHÁM PHÁ DỮ LIỆU

Trước khi đưa ra câu hỏi để trả lời, ta cần biết dữ liệu có gì. Vì vậy, đầu tiên ta cần đọc dữ liệu

In [1]:
import numpy as np
import pandas as pd
import matplotlib as plt

Dữ liệu lưu trữ thông tin của hơn 50.000 cuốn sách, có id từ 1 đến 100.000 vì vậy nên file có tên là book1-1000k.  
Để biểu diễn trực quan dữ liệu, em dùng Dataframe của pandas

In [2]:
books_df = pd.read_csv('book1-100k.csv')

Xem thử thông tin các cuốn sách trông như thế nào

In [3]:
books_df.head(2)

,Id,Name,RatingDist1,pagesNumber,RatingDist4,RatingDistTotal,PublishMonth,PublishDay,Publisher,CountsOfReview,PublishYear,Language,Authors,Rating,RatingDist2,RatingDist5,ISBN,RatingDist3
0,1,Harry Potter and the Half-Blood Prince (Harry ...,1:9896,652,4:556485,total:2298124,16,9,Scholastic Inc.,28062,2006,eng,J.K. Rowling,4.57,2:25317,5:1546466,NaN,3:159960
1,2,Harry Potter and the Order of the Phoenix (Har...,1:12455,870,4:604283,total:2358637,1,9,Scholastic Inc.,29770,2004,eng,J.K. Rowling,4.50,2:37005,5:1493113,0439358078,3:211781


In [4]:
books_df.shape

(58292, 18)

In [5]:
books_df.loc[25597, 'Name']

'The Water Babies: A Fairy Tale for a Land Baby'

Vậy dữ liệu có kích thước là 58292 dòng x 18 cột

Mỗi dòng là thông tin của một cuốn sách  
Các cột có vẻ sắp xếp khá loạn. Ví dụ như các cột RatingDist là rating của sách lại không ở gần nhau. Các cột chứa thông tin về ngày xuất bản như PublishDay, PublishMonth, PublishYear cũng như vậy. Ngoài ra, còn có sự nhầm lẫn ở tên cột PublishMonth và PublishDay bị đảo lộn  
Thông tin về các cột như sau:  
- **id**  : là id của sách trên goodreads.com
- **Name**: tên sách
- **RatingDist(x)**: rating của cuốn sách, từ 1 đến 5, có dạng `rating:số lượng`
- **Publish(x)**: thời gian xuất bản
- **Publisher**: Nhà xuất bản
- **pagesNumber**: số trang
- **CountsOfReview**: số lượt đánh giá
- **Language**: ngôn ngữ
- **Author**: tác giả sách
- **ISBN**: International Standard Book Number - Mã số tiêu chuẩn quốc tế cho sách

Tìm miền giá trị cho các thuộc tính dạng numeric hoặc datetime: **Publish, pagesNumber, CountsOfReview**

In [6]:
print('Miền giá trị của pagesNumber là: ', min(books_df['pagesNumber']),'-', max(books_df['pagesNumber']))
print('Miền giá trị của CountsOfReview là: ', min(books_df['CountsOfReview']),'-', max(books_df['CountsOfReview']))
print('Miền giá trị của PublishYear(tính theo năm) là: ', min(books_df['PublishYear']),'-', max(books_df['PublishYear']))

Miền giá trị của pagesNumber là:  0 - 23931
Miền giá trị của CountsOfReview là:  0 - 94850
Miền giá trị của PublishYear(tính theo năm) là:  162 - 3002


Miền giá trị của PublishYear trông khá lạ (162 - 3002), có thể đây là lỗi ở trang web

In [7]:
publish_year_sr = books_df['PublishYear'].sort_values()
print(publish_year_sr.head(3))
print(publish_year_sr.tail(3))

44378     162
57264    1850
38613    1900
Name: PublishYear, dtype: int64
53429    2020
24004    2020
25597    3002
Name: PublishYear, dtype: int64


Vậy chỉ có 1 row có PusblishYear là 162, và 1 row có PublishYear là 3002 nên có thể chắc đây chỉ là lỗi.  
Lỗi này chỉ có 2 dòng không đáng kể nên xem miền giá trị của PublishYear là: 1850 - 2020 vì số năm có vẻ hợp lệ hơn  
Một số cột có dữ liệu bị thiếu như: **Language, Publisher và ISBN**

Sắp xếp thứ tự các cột sao cho các cột có thông tin liên quan đến nhau sẽ ở gần nhau

In [8]:
books_df2 = books_df[['Id', 'Name', 'Authors', 'Language', 'pagesNumber', 'ISBN',
                     'RatingDist1', 'RatingDist2', 'RatingDist3', 'RatingDist4', 'RatingDist5', 'RatingDistTotal', 'Rating',
                     'CountsOfReview',
                     'PublishMonth', 'PublishDay', 'PublishYear', 'Publisher', 
                    ]]
books_df2.head(2)

,Id,Name,Authors,Language,pagesNumber,ISBN,RatingDist1,RatingDist2,RatingDist3,RatingDist4,RatingDist5,RatingDistTotal,Rating,CountsOfReview,PublishMonth,PublishDay,PublishYear,Publisher
0,1,Harry Potter and the Half-Blood Prince (Harry ...,J.K. Rowling,eng,652,NaN,1:9896,2:25317,3:159960,4:556485,5:1546466,total:2298124,4.57,28062,16,9,2006,Scholastic Inc.
1,2,Harry Potter and the Order of the Phoenix (Har...,J.K. Rowling,eng,870,0439358078,1:12455,2:37005,3:211781,4:604283,5:1493113,total:2358637,4.50,29770,1,9,2004,Scholastic Inc.


Nhìn chung các cột đã được sắp xếp có liên quan đến nhau cho dễ nhìn.  
Việc tiếp theo cần làm là đổi tên 2 cột bị nhầm tên là PublishDay và PublishMonth như đã đề cập ở trên  

In [9]:
books_df2 = books_df2.rename(columns={"PublishDay": "PublishMonth", "PublishMonth": "PublishDay"})
books_df2.head(2)

,Id,Name,Authors,Language,pagesNumber,ISBN,RatingDist1,RatingDist2,RatingDist3,RatingDist4,RatingDist5,RatingDistTotal,Rating,CountsOfReview,PublishDay,PublishMonth,PublishYear,Publisher
0,1,Harry Potter and the Half-Blood Prince (Harry ...,J.K. Rowling,eng,652,NaN,1:9896,2:25317,3:159960,4:556485,5:1546466,total:2298124,4.57,28062,16,9,2006,Scholastic Inc.
1,2,Harry Potter and the Order of the Phoenix (Har...,J.K. Rowling,eng,870,0439358078,1:12455,2:37005,3:211781,4:604283,5:1493113,total:2358637,4.50,29770,1,9,2004,Scholastic Inc.


## 3 - CÁC CÂU HỎI CẦN ĐƯỢC TRẢ LỜI TỪ DỮ LIỆU

- Tác giả nào được yêu thích nhất
- So sánh số lượng sách được phát hành theo thời gian
- Sách được yêu thích nhất từng năm và ngược lại

## 4 - TRẢ LỜI CÂU HỎI

### Tác giả nào được yêu thích nhất

Tìm tác giả được yêu thích nhất dựa theo dữ liệu đã cho, chỉ tính trong dữ liệu. Em sẽ tính trung bình rating các cuốn sách của từng tác giả và tác giả có trung bình rating cao nhất là được yêu thích nhất  
Đầu tiên tạo ra 1 Dictionary lưu thông tin tác giả và rating các cuốn sách của tác giả đó

In [52]:
author_rating_dict = {} # dict chứa thông tin tác giả và rating có dạng {'author': [rating, rating, ...], ...}

author_and_rating = books_df2.filter(items=['Authors', 'Rating']) # df gồm 2 cột là Authors và Rating

for i in range(len(author_and_rating)): # duyệt từng dòng
    author, rating = author_and_rating.loc[i, ['Authors', 'Rating']]
    if author in author_rating_dict:
        author_rating_dict[author].append(rating) 
    else:
        author_rating_dict[author] = [rating]

In [58]:
# Tạo dict chứa thông tin tác giả và trung bình rating sách
average_rating_dict = {}
for author in author_rating_dict:
    average_rating_dict[author] = np.mean(author_rating_dict[author])

Sau khi được 1 Dictionary chứa thông tin tác giả và rating trung bình thì chỉ cần tìm max của các trung bình đó thì được tác giả có rating sách cao nhất và cũng được yêu thích nhất

Sort dict theo value giảm dần thì rating trung bình cao nhất sẽ ở đầu dict

In [72]:
average_rating_dict = dict(sorted(average_rating_dict.items(), key = lambda item: item[1], reverse = True))

Lấy ra mảng các key (tên tác giả)

In [79]:
list(average_rating_dict.keys())[0]

"Phil T. O'Brien"

In [71]:
a = {
    'a': 4,
    'b': 1,
    'c': 2
}
a = dict(sorted(a.items(), key = lambda item: item[1], reverse = True))
list(a.keys())

['a', 'c', 'b']

## 5 - NHÌN LẠI QUÁ TRÌNH THỰC HIỆN

## 6 - TÀI LIỆU THAM KHẢO

- [kaggle.com](https://www.kaggle.com/) để lấy dữ liệu
- Thư viện [Pandas](https://pandas.pydata.org/pandas-docs/stable/index.html)